In [18]:
from helper_code import *
import numpy as np, os, sys, joblib
import ecg_plot
import pandas as pd
from glob import glob
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tsai.all import *
import torch
from sklearn.metrics import classification_report
import transformation_funcs as tfs
import seaborn as sns
from torchsummary import summary

In [19]:
# DATASET_NAME = "WFDB_CPSC2018_signitured"
DATASET_NAME = "WFDB_ChapmanShaoxing_signitured"
X = np.load('physionet_files/data/big_numpy_datasets/%s.npy'%DATASET_NAME, mmap_mode='c')
label_df = pd.read_csv("physionet_files/data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
y = snomedConvert(label_df)
# get diseases that exist in more than 5 percent of the samples
y=y[y.columns[y.sum()>(0.05*y.shape[0])]]
y.reset_index(drop=True)
X.shape,y.shape

((10247, 12, 5000), (10247, 8))

In [20]:
# more than 1 label, 0 labels, total num labels, 
np.sum(y.sum(axis=1)>0),np.sum(y.sum(axis=1)==0),y.sum().sum()

(9910, 337, 13979)

In [21]:
DATASET_NAME = "WFDB_CPSC2018_signitured"
X = np.load('physionet_files/data/big_numpy_datasets/%s.npy'%DATASET_NAME, mmap_mode='c')
label_df = pd.read_csv("physionet_files/data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
y = snomedConvert(label_df)
# get diseases that exist in more than 5 percent of the samples
y=y[y.columns[y.sum()>(0.05*y.shape[0])]]
y.reset_index(drop=True)
X.shape,y.shape

((6877, 12, 14000), (6877, 7))

In [22]:
# more than 1 label, 0 labels, total num labels, 
np.sum(y.sum(axis=1)>1),np.sum(y.sum(axis=1)==0),y.sum().sum()

(394, 368, 6903)

In [23]:
print(y.sum().to_latex())

\begin{tabular}{lr}
\toprule
{} &     0 \\
\midrule
right bundle branch block    &  1857 \\
ventricular ectopics         &   700 \\
atrial fibrillation          &  1221 \\
1st degree av block          &   722 \\
premature atrial contraction &   616 \\
sinus rhythm                 &   918 \\
st depression                &   869 \\
\bottomrule
\end{tabular}



In [24]:
DATASET_NAME = "WFDB_PTBXL_signitured"
X = np.load('physionet_files/data/big_numpy_datasets/%s.npy'%DATASET_NAME, mmap_mode='c')
label_df = pd.read_csv("physionet_files/data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
y = snomedConvert(label_df)
# get diseases that exist in more than 5 percent of the samples
y=y[y.columns[y.sum()>(0.05*y.shape[0])]]
y.reset_index(drop=True)
X.shape,y.shape

((21837, 12, 5000), (21837, 11))

In [25]:
# more than 1 label, 0 labels, total num labels, 
np.sum(y.sum(axis=1)>1),np.sum(y.sum(axis=1)==0),y.sum().sum()

(12093, 526, 44179)

# finding overlaps in datasets

In [202]:
import os,glob
codes =  pd.read_csv("physionet_files/data/snomed_codes.csv",sep=",")[["Dx","SNOMEDCTCode"]]
folder_path = 'physionet_files/data/big_numpy_datasets/'
for filename in glob.glob(os.path.join(folder_path, '*signitured.npy')):
  with open(filename, 'r') as f:
    print ('"'+filename.split("/")[-1].split(".")[0]+'",',end=" ")

"WFDB_Ga_signitured", "WFDB_StPetersburg_signitured", "WFDB_Ningbo_signitured", "WFDB_ChapmanShaoxing_signitured", "WFDB_PTB_signitured", "WFDB_PTBXL_signitured", "WFDB_CPSC2018_2_signitured", "WFDB_CPSC2018_signitured", 

In [142]:
datasets = ["WFDB_Ga_signitured", "WFDB_StPetersburg_signitured", "WFDB_Ningbo_signitured", "WFDB_ChapmanShaoxing_signitured", "WFDB_PTB_signitured", "WFDB_PTBXL_signitured", "WFDB_CPSC2018_2_signitured"]
labels=[]
for d in datasets:
    labels.append(pd.read_csv("physionet_files/data/%s.csv"%d).drop(columns=["headers","leads"]).columns)
    

In [234]:
x = np.concatenate(labels)
unique, counts = np.unique(x, return_counts=True)
label_counts = np.asarray((unique, counts)).T
multi_labels = label_counts[label_counts[:,1]>1]
df = pd.DataFrame([multi_labels[:,1]],columns=multi_labels[:,0])

In [235]:
snomedConvert(df.copy())

,pacing rhythm,ventricular flutter,ventricular bigeminy,prolonged qt interval,fusion beats,myocardial ischemia,myocardial infarction,left ventricular hypertrophy,ventricular ectopics,atrial fibrillation,...,nonspecific intraventricular conduction disorder,atrial tachycardia,incomplete right bundle branch block,complete right bundle branch block,wolff parkinson white pattern,brady tachy syndrome,ventricular escape beat,decreased qt interval,ventricular escape rhythm,right ventricular hypertrophy
0,3,2,5,5,3,2,7,6,3,6,...,6,4,4,4,6,2,4,2,3,5


In [237]:
dataset_occurances = []
for l in df.columns:
    for d in datasets:
        dataset = pd.read_csv("physionet_files/data/%s.csv"%d).drop(columns=["headers","leads"])
        try:
            total_count = dataset[l].sum() if dataset[l].sum()>=100 else 0
            dataset_occurances.append((l,d,total_count))
        except Exception as e:
#             print(e)
            dataset_occurances.append((l,d,0))

In [238]:
df = pd.DataFrame(dataset_occurances,columns = ["label","dataset","count"])
df = df.pivot(index="label",columns="dataset",values="count")
df.index = [codes[codes["SNOMEDCTCode"] == int(x)]["Dx"].item() for x in df.index]
df

dataset,WFDB_CPSC2018_2_signitured,WFDB_ChapmanShaoxing_signitured,WFDB_Ga_signitured,WFDB_Ningbo_signitured,WFDB_PTBXL_signitured,WFDB_PTB_signitured,WFDB_StPetersburg_signitured
pacing rhythm,0,0,0,1182,296,0,0
ventricular flutter,0,0,0,0,0,0,0
ventricular bigeminy,0,0,0,0,0,0,0
prolonged qt interval,0,0,1391,337,118,0,0
fusion beats,0,0,0,114,0,0,0
...,...,...,...,...,...,...,...
brady tachy syndrome,0,0,0,0,0,0,0
ventricular escape beat,0,0,0,0,0,0,0
decreased qt interval,0,0,0,0,0,0,0
ventricular escape rhythm,0,0,0,0,0,0,0


In [242]:
s = df.sum(axis=1)
df.loc[s.sort_values(ascending=False).index[0:20]]

dataset,WFDB_CPSC2018_2_signitured,WFDB_ChapmanShaoxing_signitured,WFDB_Ga_signitured,WFDB_Ningbo_signitured,WFDB_PTBXL_signitured,WFDB_PTB_signitured,WFDB_StPetersburg_signitured
sinus rhythm,0,1826,1752,6299,18092,0,0
sinus bradycardia,0,3889,1677,12670,637,0,0
t wave abnormal,0,1876,2306,5167,2345,0,0
sinus tachycardia,303,1568,1261,5687,826,0,0
atrial flutter,0,445,186,7615,0,0,0
left axis deviation,0,382,940,1163,5146,0,0
myocardial infarction,376,0,0,0,5261,368,0
left ventricular high voltage,0,1295,0,4106,0,0,0
s t changes,0,0,0,4232,770,0,0
nonspecific st t abnormality,1290,1158,1883,0,381,0,0
